In [1]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import syft as sy
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
import torch.optim as optim
#import tensorflow as tf

import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

C:\Users\Raffaele\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


--------------------------------------------------------------------------------------------------------------------------------
###                                      Generation of Data      -         Synthetic Data
--------------------------------------------------------------------------------------------------------------------------------

In [20]:
class data_Linear():
    def __init__(self, N, n_chanels, image_height, image_weight):
        self.N = N
        self.n_chanels = n_chanels
        self.image_height = image_height
        self.image_weight = image_weight
        self.noise = 0.7
        self.data_linear = []
    def linear_model(self, x):
        tm = np.random.randn(3, 32, 32)
        return x*tm
        
    def create(self):
        for i in range(self.N):
            x_class_i = np.random.randn(1, self.n_chanels, self.image_height, self.image_weight) * self.noise
            self.data_linear.append(self.linear_model(x_class_i))
        return torch.utils.data.TensorDataset(torch.tensor(np.vstack(self.data_linear), dtype=torch.double), torch.tensor(np.hstack(np.random.randint(2, size=self.N)), dtype=torch.double))

In [21]:
class data_Non_Linear():
    def __init__(self, N, n_chanels, image_height, image_weight, c, l):
        
        self.N = N
        self.n_chanels = n_chanels
        self.image_height = image_height
        self.image_weight = image_weight
        self.noise = 0.9
        self.data_non_linear = []
        self.c = c
        self.l = l
        
    def non_linear(self, x, c, l):
        return np.exp(-(x-c)**2/l)
        
    def create(self):
        for i in range(self.N):
                    ##adding some noise
                x_class_i = np.random.randn(1, self.n_chanels, self.image_height, self.image_weight) * self.noise
                self.data_non_linear.append(self.non_linear(x_class_i, self.c, self.l))
        return torch.utils.data.TensorDataset(torch.tensor(np.vstack(self.data_non_linear), dtype=torch.double), torch.tensor(np.hstack(np.random.randint(2, size=self.N)), dtype=torch.double))

--------------------------------------------------------------------------------------------------------------------------------
### Clients Models
--------------------------------------------------------------------------------------------------------------------------------

In [65]:
class Client1(nn.Module):
    def __init__(self, x_train, y_train):
        super(Client1, self).__init__()

        self.num_samples = x_train.shape[0]
        self.x_train = x_train
        self.labels = y_train
        self.batch = 10
        
        

        self.encoder_model = nn.Sequential(nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1,1), padding='valid', dtype=torch.float64),
                                    nn.ReLU(),
                                    nn.Conv2d(32, 64, kernel_size=(3,3), stride=(1,1), padding='valid', dtype=torch.float64),
                                    nn.ReLU()
                                    )



    
        self.latent_model = nn.Sequential(
                                nn.Conv2d(64, 30*30, kernel_size=(3,3), stride=(1,1), padding='valid', dtype=torch.float64),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=(2,2)),
                                nn.Flatten())
        
        self.decoder_model = nn.Sequential(nn.Linear(108900, 10, dtype=torch.float64),
                                           nn.Softmax(dim=1))
        
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, inputs):
        outputs_encoder = self.encoder_model(inputs)
        outputs_latent = self.latent_model(outputs_encoder)
        latent_parameters = list(self.latent_model.parameters())
        outputs_class = self.decoder_model(outputs_latent)
        return outputs_class, latent_parameters

    
    def update(self, latent_parameters):
        # Get the parameters of the latent model
        latent_params = self.latent_model.parameters()
    
        # Iterate over the parameters and update them with new values
        for param, new_value in zip(latent_params, latent_parameters):
            param.data.copy_(new_value)
    
    
    def train_step(self, inputs, labels):
        
        #Forward pass
        logits, latent_parameters = self.forward(inputs)

        #Compute the loss
        labels = labels.long()
        loss = self.loss_function(logits, labels)

        #Zero the gradients
        self.optimizer.zero_grad()

        #Backward pass
        loss.backward()

        #updare the parameters
        self.optimizer.step()

        return loss.item(), latent_parameters

In [67]:
class Client2(nn.Module):
    def __init__(self, x_train, y_train):
        super(Client2, self).__init__()

        self.num_samples = x_train.shape[0]
        self.x_train = x_train
        self.labels = y_train
        self.batch = 10
        
        

        self.encoder_model = nn.Sequential(
                                    nn.Conv2d(3, 32, kernel_size=5, stride=1, dtype=torch.float64),
                                    nn.ReLU(),
                                    nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1,1), padding='valid', dtype=torch.float64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2),
                                    nn.ReLU())



    
        self.latent_model = nn.Sequential(
                                nn.Conv2d(64, 30*30, kernel_size=(3,3), stride=(1,1), padding='valid', dtype=torch.float64),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=(2,2)),
                                nn.Flatten())
        
        self.decoder_model = nn.Sequential(nn.Linear(22500, 10, dtype=torch.float64),
                                     nn.Softmax(dim=1))
        
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, inputs):
        outputs_encoder = self.encoder_model(inputs)
        outputs_latent = self.latent_model(outputs_encoder)
        latent_parameters = list(self.latent_model.parameters())
        outputs_class = self.decoder_model(outputs_latent)
        return outputs_class, latent_parameters


    def update(self, latent_parameters):
        # Get the parameters of the latent model
        latent_params = self.latent_model.parameters()
    
        # Iterate over the parameters and update them with new values
        for param, new_value in zip(latent_params, latent_parameters):
            param.data.copy_(new_value)
    
    def train_step(self, inputs, labels):
        
        #Forward pass
        logits, latent_parameters = self.forward(inputs)

        #Compute the loss
        labels = labels.long()
        loss = self.loss_function(logits, labels)

        #Zero the gradients
        self.optimizer.zero_grad()

        #Backward pass
        loss.backward()

        #updare the parameters
        self.optimizer.step()

        return loss.item(), latent_parameters

--------------------------------------------------------------------------------------------------------------------------------
## datasets
--------------------------------------------------------------------------------------------------------------------------------

In [6]:
data_Linear = data_Linear(1000, 3, 32, 32)
dataset = data_Linear.create()

In [7]:
data_Non_Linear = data_Non_Linear(1000, 1, 28, 28, 3, 5)
dataset_non = data_Non_Linear.create()

--------------------------------------------------------------------------------------------------------------------------------
## dataLoaders
--------------------------------------------------------------------------------------------------------------------------------

In [8]:
X_train, X_test = train_test_split(dataset, test_size=0.2, random_state=42)

trainloader_linear = torch.utils.data.DataLoader(X_train, batch_size=10, shuffle=True, num_workers=2)
testloader_linear = torch.utils.data.DataLoader(X_test, batch_size=10, shuffle=False, num_workers=2)

In [9]:
X_train_n, X_test_n = train_test_split(dataset_non, test_size=0.2, random_state=42)

trainloader_non_linear = torch.utils.data.DataLoader(X_train_n, batch_size=10, shuffle=True, num_workers=2)
testloader_non_linear = torch.utils.data.DataLoader(X_test_n, batch_size=10, shuffle=False, num_workers=2)

In [10]:
for images, labels in trainloader_non_linear:
    print(images.shape, labels)
    break

torch.Size([10, 1, 28, 28]) tensor([1, 0, 0, 1, 1, 1, 1, 0, 1, 0], dtype=torch.int32)


In [68]:
client1 = Client1(x_train_linear, y_train_linear)
client2 = Client2(x_train_non_linear, y_train_non_linear)

num_epochs = 10
latent_parameters = []

#training loop
for epoch in range(num_epochs):
    
    client1.train()
    client2.train()
    
    for i, ((images_linear, labels_linear), (images_non_linear, labels_non_linear)) in enumerate(zip(trainloader_linear, trainloader_non_linear)):
        loss_1, latent_parameters_1 = client1.train_step(images_non_linear, labels_non_linear)
        loss_2, latent_parameters_2 = client2.train_step(images_linear, labels_linear)
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss_1:.4f}")
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss_2:.4f}")
    if ((epoch+1) == (epoch+1)):
        print(epoch+1)
        latent_parameters =[(x + y) / 2 for x, y in zip(latent_parameters_1, latent_parameters_2)]
        client1.update(latent_parameters)
        client2.update(latent_parameters)
    
    latent_parameters = []

Epoch [1/10], Loss: 2.1612
Epoch [1/10], Loss: 1.8611
1
Epoch [2/10], Loss: 1.7612
Epoch [2/10], Loss: 1.9612
2
Epoch [3/10], Loss: 1.9612
Epoch [3/10], Loss: 1.7612
3
Epoch [4/10], Loss: 2.0612
Epoch [4/10], Loss: 1.9612
4
Epoch [5/10], Loss: 1.7612
Epoch [5/10], Loss: 1.9612
5
Epoch [6/10], Loss: 2.1612
Epoch [6/10], Loss: 1.7612
6
Epoch [7/10], Loss: 2.0612
Epoch [7/10], Loss: 1.6612
7
Epoch [8/10], Loss: 1.9612
Epoch [8/10], Loss: 2.1611
8
Epoch [9/10], Loss: 2.2612
Epoch [9/10], Loss: 2.1611
9
Epoch [10/10], Loss: 2.1612
Epoch [10/10], Loss: 1.7612
10


In [11]:
x_train_linear = []
y_train_linear = []
x_test_linear = []
y_test_linear = []

for images, labels in trainloader_linear:
    x_train_linear.append(images)
    y_train_linear.append(labels)
    
for images_test, labels_test in testloader_linear:
    x_test_linear.append(images_test)
    y_test_linear.append(labels_test)

# Concatenate the batches to obtain the full datasets
x_train_linear = torch.cat(x_train_linear, dim=0)
y_train_linear = torch.cat(y_train_linear, dim=0)

x_test_linear = torch.cat(x_test_linear, dim=0)
y_test_linear = torch.cat(y_test_linear, dim=0)

In [12]:
x_train_non_linear = []
y_train_non_linear = []
x_test_non_linear = []
y_test_non_linear = []

for images, labels in trainloader_non_linear:
    x_train_non_linear.append(images)
    y_train_non_linear.append(labels)
    
for images_test, labels_test in testloader_non_linear:
    x_test_non_linear.append(images_test)
    y_test_non_linear.append(labels_test)

# Concatenate the batches to obtain the full datasets
x_train_non_linear = torch.cat(x_train_non_linear, dim=0)
y_train_non_linear = torch.cat(y_train_non_linear, dim=0)

x_test_non_linear = torch.cat(x_test_non_linear, dim=0)
y_test_non_linear = torch.cat(y_test_non_linear, dim=0)

In [70]:
def test(inputs_1, labels_1, inputs_2, labels_2):
        logits_1, _ = client1.forward(inputs_1)
        logits_2, _ = client2.forward(inputs_2)
        

        _, predictions_1 = torch.max(logits_1, 1)
        _, predictions_2 = torch.max(logits_2, 1)
        
        acc_1 = torch.mean((predictions_1 == labels_1).float())
        acc_2 = torch.mean((predictions_2 == labels_2).float())

        return acc_1.item(), acc_2.item() 
print(test(x_test_non_linear, y_test_non_linear, x_test_linear, y_test_linear))

(0.47999998927116394, 0.5149999856948853)
